In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pickle 
import pandas as pd

In [2]:
x_array = ['x_normal', 'x_exponential', 'x_lognormal']
y_array = ['y_normal', 'y_exponential', 'y_lognormal']

In [3]:
# fig, ax = plt.subplots(len(x_array), len(y_array), figsize=(16, 9))
# fig.tight_layout()


# activation = activation=keras.activations.relu


# for idx, x_name in enumerate(x_array):
#     for idy, y_name in enumerate(y_array):
#         cur_x, cur_y = eval(x_name), eval(y_name)
        

#         for name_transformation, func_transformation in tranformations.items():
#             transformed_x = func_transformation(cur_x)
#             try:
#                 model = MLP(transformed_x.shape[1], 1, 1024, depth=2, activation=activation)
#                 model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4), loss='mse')
#                 model.fit(transformed_x, cur_y, epochs=1000, verbose=0)
#                 y_hat = model.predict(transformed_x)
#                 score = mse(y_hat, cur_y)
#                 print(f"{score} : {name_transformation}")
#                 ax[idx, idy].plot(cur_x, y_hat, '-', alpha=0.7, linewidth=3, label=f'{score:0.3f} : {name_transformation}')
#             except:
#                 print(name_transformation, 'NANS')


#         ax[idx, idy].scatter(cur_x, cur_y, c='black', marker='x', label=f'Ground Truth', s=10)

#         ax[idx, idy].set_title(f'{x_name}/{y_name}')
#         ax[idx, idy].legend()
# plt.suptitle('MLP', y=1.02)
# plt.show()

In [4]:
with open('results/results_experiment1_1500epochs.pkl', 'rb') as f:
    x = pickle.load(f)

In [5]:
df = pd.json_normalize(x)
seed_n = 5
epochs = 1500

In [7]:
x_array = ['x_normal', 'x_exponential']
y_array = ['y_normal', 'y_exponential']
base_array = [2, 4, 8, 16, 32, 36]
params = np.array(['norm', 'encode_sign','only_integers'])
slice_from = 0
func = np.log2
baselines = ['intact', 'standardization']
mean 

for x in x_array:
    for y in y_array:
        for base in base_array:
            mask = (df['x']==x)&(df['y']==y)
            ne = df[mask&(df['params.base']==base)]
            intact = np.array(df[mask&(df['transformation_name']=="intact")]["history.loss"].to_list())
            standardization = np.array(df[mask&(df['transformation_name']=="standardization")]["history.loss"].to_list())
            for check_difference in params:
                left = params[params != check_difference]
                linewidth = 0.9
                fig, ax = plt.subplots(2, 2, figsize=(16,10))
                for idy, left_0 in enumerate([False, True]):
                    for idx, left_1 in enumerate([False, True]):
                        for loss in baselines:
                            ax[idy, idx].plot(func(eval(f'{loss}.mean(axis=0)')[slice_from:]), label=loss, alpha=0.2, linewidth=linewidth)
                        loss_0 = func(np.array(ne[(ne[f'params.{left[0]}']==left_0)&(ne[f'params.{left[1]}']==left_1)&(ne[f'params.{check_difference}']==False)]['history.loss'].to_list()).mean(axis=0)[slice_from:])
                        loss_1 = func(np.array(ne[(ne[f'params.{left[0]}']==left_0)&(ne[f'params.{left[1]}']==left_1)&(ne[f'params.{check_difference}']==True)]['history.loss'].to_list()).mean(axis=0)[slice_from:])
                        ax[idy, idx].plot(loss_0, '-', label=f'{check_difference}=False', linewidth=linewidth-0.2, alpha=0.8)
                        ax[idy, idx].plot(loss_1, '-', label=f'{check_difference}=True', linewidth=linewidth-0.2, alpha=0.8)
                        xticks = list(range(0, len(loss_0)+1, 100))
                        ax[idy, idx].set_xticks(xticks, labels=list(range(slice_from, len(loss_0)+slice_from+1, 100)))

                plt.suptitle(f'{x}/{y} base {base}')
                plt.legend()
                ax[0, 0].set_ylabel(f"{left[0]}=False")
                ax[1, 0].set_ylabel(f"{left[0]}=True")
                ax[0, 0].xaxis.set_label_position('top')
                ax[0, 1].xaxis.set_label_position('top')

                ax[0, 0].set_xlabel(f"{left[1]}=False")
                ax[0, 1].set_xlabel(f"{left[1]}=True")
                ax[1, 1].legend(bbox_to_anchor=(1.1, 1.05))

                plt.savefig(f'plots/{x}_{y}/base_{base}_{check_difference}.png')
                plt.close()

In [56]:
bools = [True, False]
linewidth = 2
baseline_step = 5

for x in x_array:
    for y in y_array:
        mask = (df['x']==x)&(df['y']==y)
        ne = df[mask&(df['params.base']!=10)]
        intact = np.array(df[mask&(df['transformation_name']=="intact")]["history.loss"].to_list())
        standardization = np.array(df[mask&(df['transformation_name']=="standardization")]["history.loss"].to_list())
        for norm in bools:
            for encode_sign in bools:
                for only_integers in bools:
                    fig, ax = plt.subplots(1, 1, figsize=(16,10))

                    # Baseline plotting
                    for loss, c, m in zip(baselines, ['0.4', '0.2'], ['o', '^']):
                        ax.plot(func(eval(f'{loss}.mean(axis=0)')[slice_from::baseline_step]), m, c=c, label=loss, linewidth=linewidth/1.1)
                    # Different bases
                    color = iter(plt.cm.summer(np.linspace(0, 1, len(base_array))))

                    for base in base_array:
                        c = next(color)
                        loss_0 = func(np.array(ne[(ne['params.base']==base)&(ne['params.norm']==norm)&(ne['params.encode_sign']==encode_sign)&(ne['params.only_integers']==only_integers)]['history.loss'].to_list()).mean(axis=0)[slice_from:])
                        ax.plot(loss_0, '-', label=f'base {base}',c=c, linewidth=linewidth-0.2, alpha=0.8)
                        xticks = list(range(0, len(loss_0)+1, 100))
                        ax.set_xticks(xticks, labels=list(range(slice_from, len(loss_0)+slice_from+1, 100)))
               

                        plt.suptitle(f'{x}/{y} norm {norm} encode_sign {encode_sign} only_integers {only_integers}')               
                    ax.legend(bbox_to_anchor=(1.1, 1.05))
                    ax.set_ylabel(f"{func.__name__} of MSE")
                    ax.set_xlabel(f"Epochs")
                    plt.savefig(f'plots/{x}_{y}/base_comparison/{norm}_{encode_sign}_{only_integers}.png')
                    plt.close()

'log2'